In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics

target_path = 'val_prediction.csv.03-04-2018_16:07:37'
dtypes = pickle.load(open('output_dtypes.pickle','rb'))

data = pd.read_csv(target_path, dtype=dtypes,parse_dates=["click_time"])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6303449 entries, 0 to 6303448
Data columns (total 23 columns):
click_time              datetime64[ns]
ip                      uint64
app                     uint16
device                  uint16
os                      uint16
channel                 uint16
is_attributed           uint8
hour                    uint8
day                     uint8
ip_day_hourcount        uint16
ip_appcount             uint16
ip_channelcount         uint16
ip_channel_appcount     uint16
ip_app_oscount          uint16
ipcount                 uint16
ip_hour_channelcount    uint16
ip_hour_oscount         uint16
ip_hour_appcount        uint16
channel_appcount        uint16
channel_oscount         uint16
channel_app_oscount     uint16
app_oscount             uint16
predict                 float64
dtypes: datetime64[ns](1), float64(1), uint16(17), uint64(1), uint8(3)
memory usage: 366.7 MB


In [40]:
def calculate_auc(val, prediction_col = 'predict'):
    #if getattr(val['predict'], "values", None) is None:
    #    return 0
    fpr, tpr, thresholds = metrics.roc_curve(val['is_attributed'].values,
        val[prediction_col].values)
    auc = metrics.auc(fpr, tpr)
    return auc

print('overall auc:', calculate_auc(data))

data['fake_predict'] = data['predict']*1.2
print('auc of fake prediction:', calculate_auc(data,'fake_predict'))

overall auc: 0.968729523171
auc of fake prediction: 0.968729523171


In [30]:
def top_n_values_of_dimensions(data, dimensions,count_col):
    d1 = data[dimensions + [count_col]+['is_attributed','predict']].\
        groupby(by=dimensions)[[count_col]].count().\
        sort_values(count_col, ascending=False).rename(columns={count_col:'_'.join(dimensions+['count'])})
    d1['_'.join(dimensions+['ratio'])] = d1['_'.join(dimensions+['count'])] / len(data)
    d1 = d1.query('_'.join(dimensions+['ratio']) + ' > 0.01')

        #n_chans = training_hist_attribution[group_by_cols + [counting_col]] \
        #    .groupby(by=group_by_cols)[[counting_col]] \
        #    .count().reset_index().rename(columns={counting_col: feature_name_added1})
    d1=d1.reset_index()
    #d1['_'.join(dimensions)] = pd.Series(['_'. \
    #    join([str(row[dimension]) for dimension in dimensions]) for index, row in d1.iterrows()])
    #data['_'.join(dimensions)] = pd.Series(['_'. \
    #    join([str(row[dimension]) for dimension in dimensions]) for index, row in data.iterrows()])
    
    #data = data.set_index('_'.join(dimensions))
    #data = data.set_index(dimensions)
    
    
    #d1['_'.join(dimensions + ['auc'])] = pd.Series([calculate_auc(data.loc['_'. \
    #    join([str(row[dimension]) for dimension in dimensions])]) for index, row in d1.iterrows()])
    
    #print('dataloc:',data.loc['178_1'])
    
    #print([row for index, row in d1.reset_index().iterrows()])   

    #print(data.loc[[[row[dimension]] for dimension in dimensions] for index, row in d1.reset_index().iterrows()])   
    #print([calculate_auc(data.loc[row[dimensions]]) for index, row in d1.iterrows()])  
    
    #print(data[[dimension for dimension in dimensions]])
    auc_list = []
    for index, row in d1.reset_index().iterrows():
        query_str = ' & '.join(['%s == \'%d\'' % (dimension, int(row[dimension])) for dimension in dimensions])
        #print('query str:', query_str)
        select_data = data.query(query_str)
        #print(select_data)
        #print('auc of select data:', calculate_auc(select_data) )
        auc_list.append(calculate_auc(select_data))
    #    l = [str(row[dimension]) for dimension in dimensions]
    #    print("l:%s:%s" % (index,'_'.join(l)))
    #    #print("data loc:", data.loc['_'.join(l)]['is_attributed'].describe())
    #    print("data loc auc: %s" % calculate_auc(data.loc['_'.join(l)]))

    d1['auc'] = pd.Series(auc_list)

    return d1

import gc
gc.collect()
print(top_n_values_of_dimensions(data, ['channel'],'app'))
print(top_n_values_of_dimensions(data, ['app'],'os'))
print(top_n_values_of_dimensions(data, ['os'],'app'))
print(top_n_values_of_dimensions(data, ['device'],'os'))
print(top_n_values_of_dimensions(data, ['ip'],'os'))
print(top_n_values_of_dimensions(data, ['hour'],'os'))

   channel  channel_count  channel_ratio       auc
0      107         364663       0.057851  0.772768
1      280         336594       0.053398  0.886332
2      477         246016       0.039029  0.731441
3      101         227930       0.036160  0.986554
4      265         187995       0.029824  0.729849
5      145         183645       0.029134  0.742323
6      153         161661       0.025646  0.825915
7      259         161527       0.025625  0.891054
8      178         160721       0.025497  0.787944
9      121         151508       0.024036  0.775549
10     134         142897       0.022670  0.773256
11     466         142369       0.022586  0.751861
12     205         140104       0.022227  0.749175
13     379         137763       0.021855  0.737700
14     232         135082       0.021430  0.791275
15     442         125632       0.019931  0.740365
16     245         103870       0.016478  0.857417
17     127         103702       0.016452  0.684062
18     489          96326      

In [31]:
data_that_ip_ever_attributed = data.groupby('ip').filter(lambda x: x['is_attributed'].max() == 1)

print('auc of all the data whose ip ever attributed:', calculate_auc(data_that_ip_ever_attributed))

auc of all the data whose ip ever attributed: 0.968926772729


In [36]:
print('mean predict:', data_that_ip_ever_attributed['predict'].mean())
print('mean attibution:', data_that_ip_ever_attributed['is_attributed'].mean())

mean predict: 0.05121659569571085
mean attibution: 0.00332186329465


In [34]:
data_that_ip_never_attributed = data.groupby('ip').filter(lambda x: x['is_attributed'].max() != 1)
print('mean positive prediction of all the data whose ip never attributed:', 
      (data_that_ip_never_attributed['predict'].mean()))

mean positive prediction of all the data whose ip never attributed: 0.048391712672144475
